In [2]:
# Tensorflow Function and Graphs

In [3]:
import tensorflow as tf
import timeit
from datetime import datetime
import numpy as np

tf.version.VERSION

'2.12.0'

In [4]:
# Tensorflow graphs
"""
In TensorFlow, two modes of execution exist: eager execution and graph execution.

1. Eager Execution: This mode executes TensorFlow operations immediately,
    operation by operation, within the Python environment. It's more intuitive
    for debugging and interactive development but can have some overhead and
    might not be as optimized for performance.

2. Graph Execution: In this mode, tensor computations are represented as a computational
    graph, where operations are nodes and tensors are edges. The entire graph is
    optimized and executed, which can lead to better performance, especially for complex
    models. Graph execution also allows for portability beyond Python, which is
    important for deployment on various platforms.

"""

"""
A TensorFlow graph is a data structure composed of two main components:

1. tf.Operation objects: These represent individual units of computation, like
   mathematical operations or layers in a neural network.
2. tf.Tensor objects: These represent data that flow between operations, essentially
   the inputs and outputs of operations.

These components are defined within a tf.Graph context, and together they form a
directed acyclic graph (DAG) representing the computation flow.
"""

'\nA TensorFlow graph is a data structure composed of two main components:\n\n1. tf.Operation objects: These represent individual units of computation, like \n   mathematical operations or layers in a neural network.\n2. tf.Tensor objects: These represent data that flow between operations, essentially \n   the inputs and outputs of operations.\n\nThese components are defined within a tf.Graph context, and together they form a \ndirected acyclic graph (DAG) representing the computation flow.\n'

In [5]:
# Benefits of using Graphs
"""
1. Portability and Flexibility:

With a TensorFlow graph, you gain significant flexibility. You can utilize your
computation graph in environments that lack a Python interpreter. This includes
scenarios like mobile applications, embedded devices, and backend servers.
TensorFlow uses graphs as the fundamental format for saving and exporting models
from Python. This portability ensures that models can be deployed and used in a
wide range of settings beyond just Python-based environments.

2. Graph Optimization:

One of the key advantages of graphs is their inherent potential for optimization.
TensorFlow's graph execution engine can perform a variety of optimizations,
leading to enhanced computational efficiency:

  1.Constant Folding: TensorFlow can statically infer the values of tensors by
    collapsing constant nodes in your computation. This process is known as constant
    folding, which reduces redundant calculations involving constants.
  2.Parallel Execution: The graph structure enables TensorFlow to identify sub-parts
    of a computation that are independent and execute them concurrently on multiple
    threads or devices. This parallelism accelerates the execution of the graph.
  3.Common Subexpression Elimination: TensorFlow's optimization system, Grappler,
    can identify and eliminate common subexpressions in the computation graph,
    simplifying arithmetic operations and further improving efficiency.

3. Grappler Optimization System:

TensorFlow features an entire optimization system known as Grappler. This system
is responsible for performing a wide range of optimizations on the computation graph,
including the ones mentioned above. Grappler aims to enhance the speed and efficiency
of TensorFlow execution across various hardware platforms.

4. Improved Performance:

In summary, graphs offer several advantages that contribute to improved performance
of TensorFlow models:

    Faster Execution: Optimization and parallelism provided by graphs lead to faster
    execution times for complex computations.
    Parallelism: The ability to identify independent sub-parts of a computation and
    execute them concurrently on different threads or devices enhances overall performance.
    Efficiency on Multiple Devices: Graphs allow TensorFlow to efficiently utilize
    multiple devices, such as GPUs and TPUs, for parallel execution.

5. Defining Models in Python:

While graphs offer numerous benefits, it's still convenient to define machine learning
models (or any computations) in Python. TensorFlow allows you to define models in Python
and then automatically constructs the corresponding computation graph when it's needed.
This provides the best of both worlds: the convenience of defining models in a high-level
language and the efficiency and optimization of graph execution.
"""

"\n1. Portability and Flexibility:\n\nWith a TensorFlow graph, you gain significant flexibility. You can utilize your \ncomputation graph in environments that lack a Python interpreter. This includes \nscenarios like mobile applications, embedded devices, and backend servers. \nTensorFlow uses graphs as the fundamental format for saving and exporting models \nfrom Python. This portability ensures that models can be deployed and used in a \nwide range of settings beyond just Python-based environments.\n\n2. Graph Optimization:\n\nOne of the key advantages of graphs is their inherent potential for optimization. \nTensorFlow's graph execution engine can perform a variety of optimizations, \nleading to enhanced computational efficiency:\n\n  1.Constant Folding: TensorFlow can statically infer the values of tensors by \n    collapsing constant nodes in your computation. This process is known as constant \n    folding, which reduces redundant calculations involving constants.\n  2.Parallel E

In [6]:
# Creating Tensorflow graphs using tf.Function wrapper
# A Function is a Python callable that builds TensorFlow graphs from the Python
# function. You use a Function in the same way as its Python equivalent.

# Define a Python function.
def some_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(some_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = some_regular_function(x1, y1, b1).numpy()
# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)  # asserts True

In [7]:
# tf.function applies to a function and all other functions it calls.

def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

In [8]:
# Note:  tf.function uses a library called AutoGraph (tf.autograph) to convert
# Python code into graph-generating code.

print("First branch, with graph:", outer_function(tf.constant([[1.0, 2.0]])).numpy())
print("Second branch, with graph:", outer_function(tf.constant([[1.0, -2.0]])).numpy())

def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph: [[12.]]
Second branch, with graph: [[0.]]
First branch, with graph: 1
Second branch, with graph: 0


In [9]:

# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu)) # only apply on open function before
                                            # converting it into graph.

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [10]:
# Lets understand how A Function encapsulates several tf.Graphs behind one API
# using polymorphism and make it efficient execution and deployable.
"""
A tf.Graph is specialized to a specific type of inputs (for example, tensors with
a specific dtype or objects with the same id()).

Each time you invoke a Function with a set of arguments that can't be handled by
any of its existing graphs (such as arguments with new dtypes or incompatible shapes),
Function creates a new tf.Graph specialized to those new arguments. The type
specification of a tf.Graph's inputs is known as its input signature or just a signature.
"""

# The most notable feature of a Function is its ability to encapsulate several tf.Graphs
# behind a single API. Each tf.Graph represents a computation graph that defines the
# operations and data flow for a specific computation. By encapsulating multiple graphs
# within a Function, you gain the ability to create a unified and optimized execution environment.

@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# `my_relu` creates new graphs as it observes more signatures.
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor([3. 0.], shape=(2,), dtype=float32)


In [11]:
# If the Function has already been called with that signature, Function does not
# create a new tf.Graph.
print(my_relu([9, -2]))   # new graph, has unique input (not a tensor, but python argument)
print(my_relu(tf.constant([9.0, 6.5])))  # no new graph, has same tensor type
print(my_relu(tf.constant(6.0)))  # no new graph, has same tensor type

# Note: New Python arguments always trigger the creation of a new graph, hence the extra tracing.
print(my_relu.pretty_printed_concrete_signatures())

tf.Tensor([9. 0.], shape=(2,), dtype=float32)
tf.Tensor([9.  6.5], shape=(2,), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
my_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

my_relu(x=[1, -1])
  Returns:
    float32 Tensor, shape=(2,)

my_relu(x)
  Args:
    x: float32 Tensor, shape=(2,)
  Returns:
    float32 Tensor, shape=(2,)

my_relu(x=[9, -2])
  Returns:
    float32 Tensor, shape=(2,)


In [12]:
# different scanrios of using tf.function
# 1. Graph execution vs. eager execution

"""
The code in a Function can be executed both eagerly and as a graph.
By default, Function executes its code as a graph
"""
# To make it execute eagerly, It can be set with tf.config.run_functions_eagerly(True)
# This is a switch that turns off Function's ability to create and run graphs,
# instead of executing the code normally.

"""
However, Function can behave differently under graph and eager execution.
The Python print function is one example of how these two modes differ. Let's check
out what happens when you insert a print statement to your function and call it repeatedly.
"""

@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

error1 = get_MSE(y_true, y_pred)
error2 = get_MSE(y_true, y_pred)
error3 = get_MSE(y_true, y_pred)
# Print is getting executed only once

Calculating MSE!


In [13]:
# The print statement is executed when Function runs the original code in order
# to create the graph in a process known as "tracing".
"""
Tracing captures the TensorFlow operations into a graph, and print is not captured in the
graph. That graph is then executed for all three calls without ever running the Python code again.
"""
# Now, globally set everything to run eagerly to force eager execution.
tf.config.run_functions_eagerly(True)

error1 = get_MSE(y_true, y_pred)
error2 = get_MSE(y_true, y_pred)
error3 = get_MSE(y_true, y_pred)
# It will print 3 times as it runs as python code not as graph
# Better solution is to use tf.print() which will be included in graph

Calculating MSE!
Calculating MSE!
Calculating MSE!


In [14]:
# Non-Strict Execution in TensorFlow: Understanding Graph Execution Behavior

"""
1. Non-Strict Execution vs. Eager Execution:

In TensorFlow, two primary execution modes exist: non-strict execution (graph execution)
and eager execution. Eager execution involves executing each operation as it appears in
the code, step by step, allowing for immediate feedback and debugging. Non-strict
execution, on the other hand, is characteristic of graph execution, where only operations
necessary to produce observable effects are executed.
"""

"""
2. Observable Effects and Non-Strict Execution:

In non-strict execution, only operations that lead to observable effects are executed.
These observable effects include:

  1. The return value of the function.
  2. Well-known side-effects, such as:
       -> Input/output operations like tf.print.
      -> Debugging operations like assertion functions in tf.debugging.
      -> Mutations of tf.Variable.

"""

"""
3. Skipping Unnecessary Operations:

Graph execution only performs operations that contribute to producing the observable
effects mentioned above. If an operation doesn't affect any of these effects, it's
skipped during graph execution, leading to potential optimizations in terms of
execution time and computational resources.

4. Runtime Error Handling:

An important aspect of non-strict execution is that runtime error checking is not
counted as an observable effect. If an operation is skipped during graph execution
because it's deemed unnecessary, it won't raise any runtime errors related to that operation.
"""
tf.config.run_functions_eagerly(False)  # make it false if it is already True

@tf.function
def unused_return_graph(x):
  tf.gather(x, [1]) # unused
  return x

# Only needed operations are run during graph execution. The error is not raised.
print(unused_return_graph(tf.constant([0.0])))


tf.Tensor([0.], shape=(1,), dtype=float32)


In [15]:
# But with Eager execution, It will show error.
def unused_return_eager(x):
  # Get index 1 will fail when `len(x) == 1`
  tf.gather(x, [1]) # unused
  return x

try:
  print(unused_return_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
  # All operations are run during eager execution so an error is raised.
  print(f'{type(e).__name__}: {e}')

InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0] = 1 is not in [0, 1) [Op:GatherV2]


In [16]:
# Speed Enhancement using graph execution:
"""
tf.function usually improves the performance of your code, but the amount of speed-up
depends on the kind of computation you run. Small computations can be dominated by
the overhead of calling a graph.

Graphs can speed up your code, but the process of creating them has some overhead.
For some functions, the creation of the graph takes more time than the execution of the graph.

Note: No matter how large your model, you want to avoid tracing frequently or multiple times.
"""

'\ntf.function usually improves the performance of your code, but the amount of speed-up \ndepends on the kind of computation you run. Small computations can be dominated by \nthe overhead of calling a graph.\n\nGraphs can speed up your code, but the process of creating them has some overhead. \nFor some functions, the creation of the graph takes more time than the execution of the graph.\n\nNote: No matter how large your model, you want to avoid tracing frequently or multiple times.\n'

In [17]:
# Understanding TensorFlow Functions tf.function in more detail
"""
The main takeaways and recommendations are:

1. Debug in eager mode, then decorate with @tf.function.
2. Don't rely on Python side effects like object mutation or list appends.
3. tf.function works best with TensorFlow ops; NumPy and Python calls are converted to constants.
"""

"\nThe main takeaways and recommendations are:\n\n1. Debug in eager mode, then decorate with @tf.function.\n2. Don't rely on Python side effects like object mutation or list appends.\n3. tf.function works best with TensorFlow ops; NumPy and Python calls are converted to constants.\n"

In [18]:
# Tracing in tensorflow Graph execution using tf.function
"""
In TensorFlow, computations are usually executed within a computational graph, which
allows for optimizations like parallel execution and device placement. However, not all
Python code can be directly represented in a TensorFlow graph due to differences in
execution models and data types.
 e.g: Raising an error, or working with a more complex Python object; none of these
      things can run in a tf.Graph
"""

# Function bridges this gap by separating your code in two stages:
"""
Tracing Stage:
   In this stage, a new TensorFlow graph (tf.Graph) is created. During tracing,
   TensorFlow operations within the function are deferred – they are captured by
   the graph and not immediately executed. Python code runs normally, but TensorFlow
   operations are recorded for later execution in the graph.

Execution Stage:
   In this stage, the traced graph is executed. This stage is much faster than the
   tracing stage because the graph is optimized and ready for efficient execution.

"""
# Depending on its inputs,, Skipping the first stage and only executing the second
# stage is what gives you TensorFlow's high performance.

# When you pass arguments of different types into a Function, both stages are run:

@tf.function
def double(a):
  print("Tracing with", a)
  return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [19]:
# Note that if you repeatedly call a Function with the same argument type,
# TensorFlow will skip the tracing stage and reuse a previously traced graph, as
# the generated graph would be identical.
print(double(tf.constant("b")))
print(double(tf.constant("c")))
print(double(tf.constant("d")))  # tracing is not called as their graph already created

tf.Tensor(b'bb', shape=(), dtype=string)
tf.Tensor(b'cc', shape=(), dtype=string)
tf.Tensor(b'dd', shape=(), dtype=string)


In [20]:
# To see all of the available traces
print(double.pretty_printed_concrete_signatures())  # 3 traces are made

double(a)
  Args:
    a: int32 Tensor, shape=()
  Returns:
    int32 Tensor, shape=()

double(a)
  Args:
    a: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


In [ ]:
#  tf.function creates a cached, dynamic dispatch layer over TensorFlow's graph tracing logic
"""
1. A tf.Graph is the raw, language-agnostic, portable representation of a TensorFlow computation.
2. A ConcreteFunction wraps a tf.Graph.
3. A Function manages a cache of ConcreteFunctions and picks the right one for your inputs.
   tf.function wraps a Python function, returning a Function object.
4. Tracing creates a tf.Graph and wraps it in a ConcreteFunction, also known as a trace.
"""

In [ ]:
# Rules of Tracing in tf.function
# Tracing is a pivotal step in the functioning of Function, as it involves creating
# and reusing ConcreteFunctions for optimal performance.
"""

1. Matching Call Arguments to ConcreteFunctions:

When a Function is called, it attempts to match the call arguments to existing
ConcreteFunctions. This matching is done using tf.types.experimental.TraceType
associated with each argument. If a matching ConcreteFunction is found, the call
is dispatched to it. If no match exists, a new ConcreteFunction is traced.

2. Subtyping and Specific Signatures:

Tracing selects the most specific signature when multiple matches are found.
Subtyping rules are employed for matching, similar to how function calls work in
languages like C++ or Java. For instance, if a TensorShape([1, 2]) argument is passed,
a ConcreteFunction with a broader shape like TensorShape([None, None]) can be matched,
but if a more specific one like TensorShape([1, None]) exists, it will be prioritized.

3. Determining TraceType for Different Inputs:

The TraceType for various input types is determined as follows:

  ->  For Tensor, the type is determined by its dtype and shape. Ranked shapes are
    subtypes of unranked shapes, and fixed dimensions are subtypes of unknown dimensions.
  ->  For Variable, the type is similar to a Tensor, but includes a unique resource
    ID of the variable to handle control dependencies correctly.
  ->  For Python primitive values, the type corresponds directly to the value itself.
    For example, the TraceType of the value 3 is LiteralTraceType<3>, not int.
  ->  For Python ordered containers like lists and tuples, the type depends on the
    types of their elements. For instance, the type of [1, 2] is ListTraceType<LiteralTraceType<1>, LiteralTraceType<2>>.
  ->  For Python mappings like dictionaries, the type is a mapping from the same
    keys to the types of values instead of the actual values themselves.
  ->  For Python objects implementing the __tf_tracing_type__ method, the type is
    determined by the return value of that method.
  ->  For other Python objects, a generic TraceType is used. The matching procedure
    involves checking if the object is the same as the one used in the previous
    trace or if it's equal to the previous trace's object.

4. Importance of Immutability:

When using Python objects as arguments to tf.function, it's recommended to use immutable objects.
The procedure for checking object identity and equality relies on this. Weak references
to the object are maintained, so the object must remain in scope and not be deleted.

"""

In [27]:
# Controlling retracing
# If your Function retraces a new graph for every call, you'll find that your code
# executes more slowly than if you didn't use tf.function.

# 1. Pass a fixed input_signature to tf.function

@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
  print("Tracing with", x)
  return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))

# You specified a 1-D tensor in the input signature, so this should fail (dim > 1).
#next_collatz(tf.constant([[1, 2], [3, 4]]))

# You specified an int32 dtype in the input signature, so this should fail.(wrong dtype)
#next_collatz(tf.constant([1.0, 2.0]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)


In [28]:
# 2. Use unknown dimensions for flexibility
# Since TensorFlow matches tensors based on their shape, using a None dimension
# as a wildcard will allow Functions to reuse traces for variably-sized input.

@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def g(x):
  print('Tracing with', x)
  return x

# No retrace!
print(g(tf.constant([1, 2, 3])))
print(g(tf.constant([1, 2, 3, 4, 5])))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


In [29]:
# 3. Pass tensors instead of python literals
# We should avoid passing python arguments, rather pass tensors.

def train_one_step():
  pass

@tf.function
def train(num_steps):
  print("Tracing with num_steps = ", num_steps)
  tf.print("Executing with num_steps = ", num_steps)
  for _ in tf.range(num_steps):
    train_one_step()

print("Retracing occurs for different Python arguments.")
train(num_steps=10)
train(num_steps=20)

print()
print("Traces are reused for Tensor arguments.")
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Retracing occurs for different Python arguments.
Tracing with num_steps =  10
Executing with num_steps =  10
Tracing with num_steps =  20
Executing with num_steps =  20

Traces are reused for Tensor arguments.
Tracing with num_steps =  Tensor("num_steps:0", shape=(), dtype=int32)
Executing with num_steps =  10
Executing with num_steps =  20


In [30]:
# If you need to force retracing, create a new Function. Separate Function objects
# are guaranteed not to share traces.
def f():
  print('Tracing!')
  tf.print('Executing')

tf.function(f)()  # new Function object
tf.function(f)()  # new Function object

Tracing!
Executing
Tracing!
Executing


In [31]:
# Obtaining concrete functions
"""
Every time a function is traced, a new concrete function is created. You can directly
obtain a concrete function, by using get_concrete_function.
"""
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.constant("a"))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [32]:
# Printing a ConcreteFunction displays a summary of its input arguments (with types) and its output type.
print(double_strings)

ConcreteFunction double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


In [33]:
# Obtaining graphs
# Each concrete function is a callable wrapper around a tf.Graph
graph = double_strings.graph
for node in graph.as_graph_def().node:
  print(f'{node.input} -> {node.name}')

[] -> a
['a', 'a'] -> add
['add'] -> Identity


In [ ]:
# Debugging
"""
In general, debugging code is easier in eager mode than inside tf.function. You should
ensure that your code executes error-free in eager mode before decorating with tf.function.
"""
# To assist in the debugging process, you can call tf.config.run_functions_eagerly(True)
# to globally disable and reenable tf.function.

"""
When tracking down issues that only appear within tf.function, here are some tips:

-> Plain old Python print calls only execute during tracing, helping you track down
  when your function gets (re)traced.
-> tf.print calls will execute every time, and can help you track down intermediate
  values during execution.
-> tf.debugging.enable_check_numerics is an easy way to track down where NaNs and Inf are created.
-> pdb (the Python debugger) can help you understand what's going on during tracing.
  (Caveat: pdb will drop you into AutoGraph-transformed source code.)

In [46]:
# AutoGraph is a library of TensorFlow that automatically converts certain Python
# constructs, like loops and conditionals, into equivalent TensorFlow operations.
# When working with conditionals, AutoGraph has a distinct behavior depending on
# whether the condition is a TensorFlow Tensor or a Python value.

@tf.function
def fizzbuzz(n):
  for i in tf.range(1, n + 1):
    print('Tracing for loop')
    if i % 15 == 0:
      print('Tracing fizzbuzz branch')
      tf.print('fizzbuzz')
    elif i % 3 == 0:
      print('Tracing fizz branch')
      tf.print('fizz')
    elif i % 5 == 0:
      print('Tracing buzz branch')
      tf.print('buzz')
    else:
      print('Tracing default branch')
      tf.print(i)

fizzbuzz(tf.constant(5))
print()
fizzbuzz(tf.constant(2))

"""
tf.cond is a TensorFlow operation that traces and adds all branches of a conditional
to the graph. That is why print statement will print from each branch at graph creation.
"""

Tracing for loop
Tracing fizzbuzz branch
Tracing fizz branch
Tracing buzz branch
Tracing default branch
1
2
fizz
4
buzz

1
2


'\ntf.cond is a TensorFlow operation that traces and adds all branches of a conditional \nto the graph. That is why print statement will print from each branch at graph creation.\n'

In [48]:
# Similarily, we have tf.while_loop for for/while loop of python
"""
A Python loop executes during tracing, adding additional ops to the tf.Graph for
every iteration of the loop.

A TensorFlow loop traces the body of the loop, and dynamically selects how many
iterations to run at execution time. The loop body only appears once in the generated tf.Graph.
"""

@tf.function
def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

f(tf.random.uniform([2]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.27705026, 0.0154227 ], dtype=float32)>

In [ ]:
print(tf.autograph.to_code(fizzbuzz.python_function))

In [ ]:
# common Pitfall of using tf.function on Loop functions
"""
A common pitfall is to loop over Python/NumPy data within a tf.function. This loop will
execute during the tracing process, adding a copy of your model to the tf.Graph for each
iteration of the loop.

If you want to wrap the entire training loop in tf.function, the safest way to do this is
to wrap your data as a tf.data.Dataset so that AutoGraph will dynamically unroll the
training loop.

"""
# Reading data from files via TFRecordDataset, CsvDataset, etc. is the most effective
# way to consume data, as then TensorFlow itself can manage the asynchronous loading
# and prefetching of data, without having to involve Python.

In [ ]:
# Limitations of using tf.function

# TensorFlow Function has a few limitations by design that you should be aware of
# when converting a Python function to a Function.

"""
1. Executing Python side effects
Side effects, like printing, appending to lists, and mutating globals, can behave
unexpectedly inside a Function, sometimes executing twice or not all. They only happen
the first time you call a Function with a set of inputs. Afterwards, the traced
tf.Graph is reexecuted, without executing the Python code.

-> The general rule of thumb is to avoid relying on Python side effects in your logic
   TensorFlow APIs like tf.data, tf.print, tf.summary, tf.Variable.assign, and tf.TensorArray
   are the best way to ensure your code will be executed by the TensorFlow runtime with each call.
-> If you would like to execute Python code during each invocation of a Function, tf.py_function
   is an exit hatch. The drawback of tf.py_function is that it's not portable or particularly
   performant, cannot be saved with SavedModel, and does not work well in distributed
   (multi-GPU, TPU) setups. Also, since tf.py_function has to be wired into the graph,
   it casts all inputs/outputs to tensors.
"""
   #Input needs to be reshaped properly at the end (common problem in Computer Vision)

In [52]:
# 2. Changing Python global and free variables
# Changing Python global and free variables counts as a Python side effect, so it
# only happens during tracing.

external_list = []

@tf.function
def side_effect(x):
  print('Python side effect')
  external_list.append(x)


side_effect(1)
side_effect(1)
side_effect(1)
# The list append only happened once!
print(len(external_list)) # items are added only once

# In summary, as a rule of thumb, you should avoid mutating python objects such as
# integers or containers like lists that live outside the Function. Instead, use
# arguments and TF objects.

Python side effect
1


In [ ]:
# Problem with Python Iterators and Generators
# Many Python features, such as generators and iterators, rely on the Python runtime
# to keep track of state. In general, while these constructs work as expected in
# eager mode, they are examples of Python side effects and therefore only happen
# during tracing.

# Better use the tf.data API can help implement generator patterns
# => tf.data.Dataset.from_tensor_slices

In [53]:
# All outputs of a tf.function must be return values
"""
With the exception of tf.Variables, a tf.function must return all its outputs.
Attempting to directly access any tensors from a function without going through
return values causes "leaks".
"""
# This is true even if the leaked value is also returned:

x = None

@tf.function
def leaky_function(a):
  global x
  x = a + 1  # Bad - leaks local tensor
  return a + 2

correct_a = leaky_function(tf.constant(1))

print(correct_a.numpy())  # Good - value obtained from function's returns
try:
  x.numpy()  # Bad - tensor leaked from inside the function, cannot be used here
except AttributeError as expected:
  print(expected)

3
'Tensor' object has no attribute 'numpy'


In [54]:
"""
Usually, leaks such as these occur when you use Python statements or data structures.
In addition to leaking inaccessible tensors, such statements are also likely wrong
because they count as Python side effects, and are not guaranteed to execute at
every function call.

Common ways to leak local tensors also include mutating an external Python collection, or an object:
"""
class MyClass:

  def __init__(self):
    self.field = None

external_list = []
external_object = MyClass()

def leaky_function():
  a = tf.constant(1)
  external_list.append(a)  # Bad - leaks tensor
  external_object.field = a  # Bad - leaks tensor

In [ ]:
# Recursive tf.functions are not supported
# Recursive Functions are not supported and could cause infinite loops.

In [ ]:
# Known Issues in tf.function
# refer here: https://www.tensorflow.org/guide/function#known_issues